In [0]:
df = spark.read.table("dev_catalog.bronze.sales")

In [0]:
df_final = df.dropna()
df_final.count()

In [0]:
df_final = df.dropDuplicates().dropna().drop("ingestion_date")
df_final.count()

In [0]:
df_final.write.mode("overwrite").saveAsTable("dev_catalog.silver.sales_cleaned") 

In [0]:
%sql
create or replace table dev_catalog.silver.produts_scd(product_id int, product_name string, product_category string, product_price double)

In [0]:
%sql
MERGE INTO dev_catalog.silver.products_scd AS t
USING dev_catalog.bronze.product AS s
ON t.product_id = s.product_id
WHEN MATCHED THEN
  UPDATE SET
    t.product_name = s.product_name,
    t.product_category = s.product_category,
    t.product_price = s.product_price
WHEN NOT MATCHED THEN
  INSERT (product_id, product_name, product_category, product_price)
  VALUES (s.product_id, s.product_name, s.product_category, s.product_price)

In [0]:
%sql
WITH deduplicated_source AS ( 
SELECT 
s.*, 
ROW_NUMBER() OVER (PARTITION BY product_id ORDER BY seqNum DESC) as row_num 
FROM dev_catalog.bronze.product s 
) 
SELECT *
FROM deduplicated_source
WHERE row_num = 1

In [0]:
%sql 

CREATE OR REPLACE TABLE dev_catalog.silver.products_scd (product_id int, product_name string, product_category string, product_price double) 

 

In [0]:
%sql
WITH deduplicated_source AS ( 
SELECT 
s.*, 
ROW_NUMBER() OVER (PARTITION BY product_id ORDER BY seqNum DESC) as row_num 
FROM dev_catalog.bronze.product s 
) 
MERGE INTO dev_catalog.silver.products_scd AS t
USING dev_catalog.bronze.product AS s
ON t.product_id = s.product_id
WHEN MATCHED THEN
  UPDATE SET
    t.product_name = s.product_name,
    t.product_category = s.product_category,
    t.product_price = s.product_price
WHEN NOT MATCHED THEN
  INSERT (product_id, product_name, product_category, product_price)
  VALUES (s.product_id, s.product_name, s.product_category, s.product_price)